# 🏛️ Data Architecture Patterns & Decision Making

Bienvenue dans ce module où tu vas apprendre à **concevoir des architectures data** et à **prendre des décisions techniques éclairées**. C'est le passage du "je sais coder" à "je sais architecturer".

---

## Prérequis

| Niveau | Compétence |
|--------|------------|
| ✅ Requis | Tous les modules précédents (M01-M33) |
| ✅ Requis | Expérience sur des projets data réels |
| 💡 Recommandé | Avoir été confronté à des choix d'architecture |

## 🎯 Objectifs du module

À la fin de ce module, tu seras capable de :

- Connaître les **patterns d'architecture data** majeurs
- Appliquer un **framework de décision** structuré
- Rédiger des **Architecture Decision Records (ADR)**
- Faire du **capacity planning** réaliste
- Éviter les **anti-patterns** classiques
- Adapter l'architecture au **contexte** (startup vs enterprise)

---

## 1. Introduction : L'Art de l'Architecture Data

### 1.1 Qu'est-ce qu'un Architecte Data ?

```text
┌─────────────────────────────────────────────────────────────────────────────┐
│                    ÉVOLUTION DU DATA ENGINEER                               │
│                                                                             │
│   Junior               Senior               Staff/Principal                 │
│   ──────               ──────               ───────────────                 │
│                                                                             │
│   "Comment faire"      "Quoi faire"         "Pourquoi faire"               │
│                                                                             │
│   • Exécute les        • Conçoit les        • Définit la vision            │
│     tâches               pipelines          • Prend les décisions          │
│   • Suit les           • Choisit les          d'architecture               │
│     patterns             outils             • Anticipe les                 │
│   • Apprend            • Résout les           problèmes futurs             │
│                          problèmes          • Influence l'org              │
│                          complexes                                          │
│                                                                             │
│   Focus: Code          Focus: Système       Focus: Stratégie               │
└─────────────────────────────────────────────────────────────────────────────┘
```

### 1.2 Les Questions Fondamentales

Avant tout choix technique, pose-toi ces questions :

| Question | Pourquoi c'est important |
|----------|-------------------------|
| **Quel problème résout-on ?** | Éviter le solutionnisme technologique |
| **Quelles sont les contraintes ?** | Budget, équipe, deadline, legacy |
| **Quelle est l'échelle ?** | 1 GB vs 1 PB = architectures très différentes |
| **Quelle latence est acceptable ?** | Batch vs Streaming |
| **Qui va maintenir ?** | Complexité vs compétences équipe |
| **Comment ça va évoluer ?** | Anticipation vs over-engineering |

### 1.3 Le Principe de la Solution la Plus Simple

```text
┌─────────────────────────────────────────────────────────────────────────────┐
│                    COMPLEXITÉ vs VALEUR                                     │
│                                                                             │
│   Valeur                                                                    │
│   Business    │                      ┌──── Zone Danger                     │
│       ▲       │                     ╱      (over-engineering)              │
│       │       │         ┌──────────╱                                       │
│       │       │        ╱                                                   │
│       │       │       ╱   Zone Optimale                                    │
│       │       │      ╱    (bon trade-off)                                  │
│       │       │     ╱                                                      │
│       │       │────╱                                                       │
│       │       │   ╱  Zone Simple                                           │
│       │       │  ╱   (MVP, quick wins)                                     │
│       │       └──────────────────────────────────────────▶                 │
│                              Complexité Technique                          │
│                                                                             │
│   RÈGLE : Commence simple, complexifie seulement quand NÉCESSAIRE         │
└─────────────────────────────────────────────────────────────────────────────┘
```

---

## 2. Patterns d'Architecture Data

### 2.1 Lambda vs Kappa vs Delta Architecture

```text
┌─────────────────────────────────────────────────────────────────────────────┐
│                    LAMBDA ARCHITECTURE                                      │
│                                                                             │
│                         ┌──────────────────┐                               │
│                         │   Batch Layer    │                               │
│                    ┌───▶│   (Spark Batch)  │───┐                           │
│   ┌─────────┐     │    └──────────────────┘   │    ┌─────────────────┐    │
│   │  Source │─────┤                           ├───▶│  Serving Layer  │    │
│   └─────────┘     │    ┌──────────────────┐   │    └─────────────────┘    │
│                    └───▶│   Speed Layer    │───┘                           │
│                         │   (Streaming)    │                               │
│                         └──────────────────┘                               │
│                                                                             │
│   ✅ Accurate (batch) + Fast (streaming)                                   │
│   ❌ 2 codebases à maintenir                                               │
│   ❌ Complexité opérationnelle                                             │
└─────────────────────────────────────────────────────────────────────────────┘

┌─────────────────────────────────────────────────────────────────────────────┐
│                    KAPPA ARCHITECTURE                                       │
│                                                                             │
│   ┌─────────┐     ┌──────────────────┐     ┌─────────────────┐            │
│   │  Source │────▶│   Stream Layer   │────▶│  Serving Layer  │            │
│   └─────────┘     │   (Kafka + Flink)│     └─────────────────┘            │
│                   └──────────────────┘                                     │
│                                                                             │
│   ✅ Une seule codebase                                                    │
│   ✅ Simplicité opérationnelle                                             │
│   ❌ Reprocessing = rejouer tout le stream                                 │
│   ❌ Pas idéal pour ML training (besoin de batch)                          │
└─────────────────────────────────────────────────────────────────────────────┘

┌─────────────────────────────────────────────────────────────────────────────┐
│                    DELTA ARCHITECTURE (Lakehouse)                           │
│                                                                             │
│   ┌─────────┐     ┌──────────────────┐     ┌─────────────────┐            │
│   │  Source │────▶│   Unified Layer  │────▶│  Serving Layer  │            │
│   └─────────┘     │   (Delta Lake)   │     └─────────────────┘            │
│                   │                  │                                     │
│                   │  Batch + Stream  │                                     │
│                   │  Same code/table │                                     │
│                   └──────────────────┘                                     │
│                                                                             │
│   ✅ Une seule codebase                                                    │
│   ✅ Time travel pour reprocessing                                         │
│   ✅ ACID transactions                                                     │
│   ✅ Batch et streaming sur mêmes tables                                   │
└─────────────────────────────────────────────────────────────────────────────┘
```

### 2.2 Matrice de Décision : Lambda vs Kappa vs Delta

| Critère | Lambda | Kappa | Delta |
|---------|--------|-------|-------|
| **Complexité ops** | 🔴 Haute | 🟢 Basse | 🟢 Basse |
| **Reprocessing** | 🟢 Facile | 🔴 Difficile | 🟢 Facile |
| **Latence** | 🟡 Mixte | 🟢 Temps réel | 🟡 Near real-time |
| **ML Training** | 🟢 Bon | 🔴 Difficile | 🟢 Bon |
| **Coût** | 🔴 Élevé | 🟢 Modéré | 🟢 Modéré |
| **Quand choisir** | Legacy, besoins spécifiques | Pure streaming | Recommandé par défaut |

### 2.3 Batch vs Streaming vs Hybrid

```text
┌─────────────────────────────────────────────────────────────────────────────┐
│                    QUAND UTILISER QUOI ?                                    │
│                                                                             │
│   BATCH                          STREAMING                  HYBRID         │
│   ─────                          ─────────                  ──────         │
│                                                                             │
│   • Reports quotidiens           • Fraud detection          • La plupart   │
│   • ML training                  • Real-time reco             des cas      │
│   • Data warehouse refresh       • Alerting                               │
│   • Backfill historique          • IoT monitoring          Batch pour      │
│                                  • Live dashboards          historique,    │
│   Latence: heures               Latence: secondes          Streaming pour  │
│                                                              temps réel    │
│                                                                             │
│   💡 RÈGLE : Si "temps réel" n'est pas un VRAI besoin business,           │
│              commence par BATCH. C'est plus simple et moins cher.          │
└─────────────────────────────────────────────────────────────────────────────┘
```

### 2.4 Questions pour Choisir Batch vs Streaming

| Question | Si OUI → | Si NON → |
|----------|----------|----------|
| Le business perd de l'argent si latence > 1 min ? | Streaming | Batch |
| Les données arrivent en continu 24/7 ? | Streaming | Batch |
| L'équipe a de l'expérience streaming ? | Streaming OK | Batch d'abord |
| Budget ops illimité ? | Streaming OK | Batch moins cher |
| Use case = alerting/fraud/monitoring ? | Streaming | Batch souvent OK |

### 2.5 Data Lake vs Data Warehouse vs Lakehouse

```text
┌─────────────────────────────────────────────────────────────────────────────┐
│                    DATA LAKE vs WAREHOUSE vs LAKEHOUSE                      │
│                                                                             │
│   DATA LAKE                DATA WAREHOUSE           LAKEHOUSE               │
│   ─────────                ──────────────           ─────────               │
│                                                                             │
│   ┌─────────────┐         ┌─────────────┐         ┌─────────────┐          │
│   │   S3/GCS    │         │  Snowflake  │         │ Delta Lake  │          │
│   │   (files)   │         │  Redshift   │         │  Iceberg    │          │
│   │             │         │  BigQuery   │         │             │          │
│   │ Raw + Semi  │         │  Structured │         │   Unified   │          │
│   │ structured  │         │    only     │         │             │          │
│   └─────────────┘         └─────────────┘         └─────────────┘          │
│                                                                             │
│   ✅ Cheap storage        ✅ Fast queries          ✅ Best of both          │
│   ✅ Flexible             ✅ ACID                  ✅ ACID                  │
│   ✅ All data types       ✅ Governance            ✅ Open formats          │
│   ❌ No ACID              ❌ Expensive             ✅ ML + Analytics        │
│   ❌ Query perf           ❌ Vendor lock-in        ✅ Streaming             │
│   ❌ Data swamp risk      ❌ ETL required                                   │
│                                                                             │
│   📅 2010s                📅 2000s-now             📅 2020s+                │
└─────────────────────────────────────────────────────────────────────────────┘
```

### 2.6 Recommandation Moderne

```text
┌─────────────────────────────────────────────────────────────────────────────┐
│                    RECOMMANDATION 2024+                                     │
│                                                                             │
│   DÉFAUT : Lakehouse (Delta/Iceberg) + Compute séparé (Spark/Trino)        │
│                                                                             │
│   EXCEPTIONS :                                                              │
│                                                                             │
│   • Équipe 100% SQL, pas de ML → Cloud DW (Snowflake, BigQuery)            │
│   • Budget serré, petit volume → DuckDB + Parquet                          │
│   • Enterprise legacy → Conserver DW existant, ajouter Lake                │
│   • Real-time OLAP → ClickHouse/Druid en complément                        │
└─────────────────────────────────────────────────────────────────────────────┘
```

### 2.7 Patterns de Data Modeling

```text
┌─────────────────────────────────────────────────────────────────────────────┐
│                    MEDALLION ARCHITECTURE                                   │
│                                                                             │
│   ┌─────────────┐     ┌─────────────┐     ┌─────────────┐                  │
│   │   BRONZE    │────▶│   SILVER    │────▶│    GOLD     │                  │
│   │   (Raw)     │     │  (Cleaned)  │     │ (Aggregated)│                  │
│   └─────────────┘     └─────────────┘     └─────────────┘                  │
│                                                                             │
│   • Ingestion brute    • Déduplication     • Business metrics              │
│   • Append-only        • Validation        • Dimensional models            │
│   • Schemaless OK      • Enrichissement    • Ready for BI                  │
│                        • Schema enforced                                    │
└─────────────────────────────────────────────────────────────────────────────┘

┌─────────────────────────────────────────────────────────────────────────────┐
│                    DIMENSIONAL MODELING (Kimball)                           │
│                                                                             │
│                        ┌───────────────┐                                   │
│                        │   dim_date    │                                   │
│                        └───────┬───────┘                                   │
│   ┌───────────────┐           │           ┌───────────────┐               │
│   │ dim_customer  │───────────┼───────────│  dim_product  │               │
│   └───────────────┘           │           └───────────────┘               │
│                        ┌──────┴──────┐                                     │
│                        │ fact_sales  │                                     │
│                        └─────────────┘                                     │
│                                                                             │
│   ✅ Optimisé pour BI/reporting                                            │
│   ✅ Facile à comprendre                                                   │
│   ❌ Redondance (dénormalisation)                                          │
└─────────────────────────────────────────────────────────────────────────────┘

┌─────────────────────────────────────────────────────────────────────────────┐
│                    DATA VAULT                                               │
│                                                                             │
│   ┌─────────┐     ┌─────────┐     ┌─────────┐                              │
│   │   HUB   │────▶│  LINK   │◀────│   HUB   │                              │
│   │(Customer)│     │ (Order) │     │(Product)│                              │
│   └────┬────┘     └─────────┘     └────┬────┘                              │
│        │                               │                                    │
│   ┌────▼────┐                     ┌────▼────┐                              │
│   │SATELLITE│                     │SATELLITE│                              │
│   │(attrs)  │                     │(attrs)  │                              │
│   └─────────┘                     └─────────┘                              │
│                                                                             │
│   ✅ Historisation complète                                                │
│   ✅ Flexibilité (ajout sources facile)                                    │
│   ❌ Complexe à implémenter                                                │
│   ❌ Queries plus complexes                                                │
└─────────────────────────────────────────────────────────────────────────────┘
```

### 2.8 Quand Utiliser Quel Modeling ?

| Pattern | Quand l'utiliser | Quand éviter |
|---------|------------------|---------------|
| **Medallion** | Lakehouse, ETL moderne, ML | Petit projet sans layers |
| **Dimensional (Kimball)** | BI/Reporting, DW classique | Data science, ML features |
| **Data Vault** | Enterprise, audit strict, multi-sources | Startup, MVP, équipe petite |
| **One Big Table (OBT)** | Analytics simple, ML features | Beaucoup de dimensions |

---

## 3. Framework de Décision

### 3.1 Le Process de Décision

```text
┌─────────────────────────────────────────────────────────────────────────────┐
│                    FRAMEWORK DE DÉCISION ARCHITECTURE                       │
│                                                                             │
│   1. DEFINE          2. DISCOVER         3. DECIDE          4. DOCUMENT    │
│   ──────────         ──────────          ────────           ─────────      │
│                                                                             │
│   ┌──────────┐      ┌──────────┐       ┌──────────┐       ┌──────────┐    │
│   │ Quel     │      │ Quelles  │       │ Évaluer  │       │ ADR      │    │
│   │ problème │─────▶│ options  │──────▶│ trade-   │──────▶│ (écrit)  │    │
│   │ ?        │      │ ?        │       │ offs     │       │          │    │
│   └──────────┘      └──────────┘       └──────────┘       └──────────┘    │
│                                                                             │
│   • Requirements     • 3-5 options      • Matrice          • Contexte      │
│   • Constraints      • Recherche        • PoC si doute     • Décision      │
│   • Success criteria • Benchmarks       • Consensus        • Conséquences  │
└─────────────────────────────────────────────────────────────────────────────┘
```

### 3.2 Template de Requirements

In [ ]:
# Template de Requirements pour une décision d'architecture

requirements_template = """
# ═══════════════════════════════════════════════════════════════
# REQUIREMENTS DOCUMENT
# ═══════════════════════════════════════════════════════════════

## 1. Contexte
- Projet : [Nom du projet]
- Date : [Date]
- Auteur : [Ton nom]
- Stakeholders : [Qui est impacté]

## 2. Problème à résoudre
[Description claire du problème en 2-3 phrases]

## 3. Functional Requirements (FR)
- FR1 : [Ce que le système DOIT faire]
- FR2 : [Ce que le système DOIT faire]
- FR3 : [...]

## 4. Non-Functional Requirements (NFR)

### Performance
- Latence max : [X secondes/minutes]
- Throughput : [X events/sec ou X GB/jour]
- Concurrent users : [X]

### Scalabilité
- Volume actuel : [X GB]
- Volume dans 1 an : [X GB]
- Volume dans 3 ans : [X GB]

### Disponibilité
- SLA cible : [99.9% ?]
- Downtime acceptable : [X heures/mois]
- RTO (Recovery Time) : [X heures]
- RPO (Data Loss) : [X minutes]

### Sécurité
- Données sensibles : [Oui/Non]
- Compliance : [GDPR, HIPAA, SOC2...]
- Encryption : [At rest, in transit]

## 5. Contraintes
- Budget : [X €/mois]
- Timeline : [Date de livraison]
- Équipe : [X personnes, compétences]
- Legacy : [Systèmes existants à intégrer]
- Vendor : [Restrictions cloud/vendor]

## 6. Critères de succès
- [ ] [Critère mesurable 1]
- [ ] [Critère mesurable 2]
- [ ] [Critère mesurable 3]
"""

print(requirements_template)

### 3.3 Matrice de Décision

In [ ]:
import pandas as pd

# Exemple : Choisir un orchestrateur

criteria = {
    'Critère': [
        'Facilité d\'apprentissage',
        'Scalabilité',
        'Communauté/Support',
        'Coût opérationnel',
        'Intégration cloud',
        'Fonctionnalités avancées'
    ],
    'Poids': [3, 4, 3, 4, 3, 2],  # Importance 1-5
    'Airflow': [3, 5, 5, 3, 4, 5],  # Score 1-5
    'Prefect': [4, 4, 3, 4, 3, 4],
    'Dagster': [3, 4, 3, 4, 3, 5],
    'Step Functions': [4, 5, 4, 5, 5, 3]
}

df = pd.DataFrame(criteria)

# Calcul des scores pondérés
for option in ['Airflow', 'Prefect', 'Dagster', 'Step Functions']:
    df[f'{option}_weighted'] = df['Poids'] * df[option]

# Totaux
totals = {
    'Option': ['Airflow', 'Prefect', 'Dagster', 'Step Functions'],
    'Score Total': [
        df['Airflow_weighted'].sum(),
        df['Prefect_weighted'].sum(),
        df['Dagster_weighted'].sum(),
        df['Step Functions_weighted'].sum()
    ]
}

print("📊 MATRICE DE DÉCISION : Choix d'Orchestrateur")
print("="*60)
print(df[['Critère', 'Poids', 'Airflow', 'Prefect', 'Dagster', 'Step Functions']].to_string(index=False))
print("\n📈 SCORES TOTAUX (pondérés) :")
print("-"*40)
for opt, score in zip(totals['Option'], totals['Score Total']):
    print(f"  {opt:20} : {score}")
print("\n✅ Recommandation : " + totals['Option'][totals['Score Total'].index(max(totals['Score Total']))])

---

## 4. Architecture Decision Records (ADR)

### 4.1 Pourquoi Documenter les Décisions ?

```text
┌─────────────────────────────────────────────────────────────────────────────┐
│                    PROBLÈME CLASSIQUE                                       │
│                                                                             │
│   Nouveau développeur : "Pourquoi on utilise Kafka et pas RabbitMQ ?"      │
│                                                                             │
│   Équipe : "Euh... c'était là quand je suis arrivé."                       │
│            "Je crois que c'est Jean qui a décidé, mais il est parti."      │
│            "On a toujours fait comme ça."                                  │
│                                                                             │
│   RÉSULTAT :                                                                │
│   • Décisions remises en question sans contexte                            │
│   • Mêmes erreurs répétées                                                 │
│   • Perte de temps à rediscuter                                            │
│                                                                             │
│   SOLUTION : Architecture Decision Records (ADR)                            │
└─────────────────────────────────────────────────────────────────────────────┘
```

### 4.2 Format ADR

In [ ]:
# Template ADR (Architecture Decision Record)

adr_template = """
# ADR-001 : [Titre de la décision]

## Status
[Proposed | Accepted | Deprecated | Superseded by ADR-XXX]

## Date
2024-XX-XX

## Context
[Quel est le problème ? Pourquoi cette décision est nécessaire ?]

## Decision Drivers
- [Driver 1 : ex. "Besoin de traiter 10K events/sec"]
- [Driver 2 : ex. "Équipe familière avec Python"]
- [Driver 3 : ex. "Budget limité à X€/mois"]

## Considered Options
1. [Option A]
2. [Option B]
3. [Option C]

## Decision
[Quelle option a été choisie et POURQUOI]

## Consequences

### Positives
- [Conséquence positive 1]
- [Conséquence positive 2]

### Negatives
- [Conséquence négative 1]
- [Trade-off accepté]

### Risks
- [Risque identifié et mitigation]

## Related
- [Lien vers ADR connexes]
- [Lien vers documentation]
"""

print(adr_template)

In [ ]:
# Exemple concret d'ADR

adr_example = """
# ADR-003 : Choix de Delta Lake comme Table Format

## Status
Accepted

## Date
2024-03-15

## Context
Notre Data Lake sur S3 souffre de plusieurs problèmes :
- Pas de transactions ACID (corruptions lors d'échecs de jobs)
- Pas de schema enforcement (colonnes manquantes non détectées)
- Updates/deletes impossibles (GDPR compliance)
- Pas de time travel pour debug

Nous devons choisir un table format pour résoudre ces problèmes.

## Decision Drivers
- Besoin d'ACID pour éviter les corruptions
- Support GDPR (delete user data)
- Équipe utilise déjà Spark 3.x
- Pas de budget pour Databricks
- Préférence pour l'open source

## Considered Options
1. **Delta Lake (OSS)** - Format Databricks, maintenant open source
2. **Apache Iceberg** - Format Netflix, très actif
3. **Apache Hudi** - Format Uber, orienté CDC
4. **Rester sur Parquet brut** - Status quo

## Decision
Nous choisissons **Delta Lake (Open Source)** car :

1. Meilleure intégration Spark (notre stack principale)
2. Documentation extensive et communauté active
3. Migration facile depuis Parquet (CONVERT TO DELTA)
4. Fonctionnalités matures (MERGE, time travel, schema evolution)
5. Pas de dépendance à Databricks (version OSS suffit)

Iceberg était un close second, mais l'écosystème Spark+Delta
est plus mature en 2024.

## Consequences

### Positives
- ACID transactions → plus de corruptions
- Time travel → debug et audit facilités
- Schema enforcement → erreurs détectées tôt
- MERGE → GDPR compliance possible

### Negatives
- Légère courbe d'apprentissage équipe
- Vacuum à scheduler régulièrement
- Pas de support Trino natif (workaround existe)

### Risks
- Databricks pourrait réduire investissement OSS
  → Mitigation : Iceberg comme plan B, migration possible

## Related
- ADR-001 : Choix de S3 comme storage
- ADR-002 : Choix de Spark comme compute engine
"""

print(adr_example)

### 4.3 Organisation des ADRs

```text
docs/
└── architecture/
    └── decisions/
        ├── README.md           # Index des ADRs
        ├── 0001-use-s3-storage.md
        ├── 0002-choose-spark.md
        ├── 0003-delta-lake.md
        ├── 0004-airflow-orchestration.md
        └── template.md         # Template pour nouvelles ADRs
```

### 4.4 Bonnes Pratiques ADR

| ✅ DO | ❌ DON'T |
|-------|----------|
| Écrire au moment de la décision | Écrire des mois après |
| Inclure le contexte complet | Supposer que le lecteur sait |
| Documenter les options rejetées | Ne documenter que le choix final |
| Lister les trade-offs acceptés | Prétendre que c'est parfait |
| Mettre à jour le status | Laisser des ADRs obsolètes actifs |

---

## 5. Sizing & Capacity Planning

### 5.1 Estimer le Storage

In [ ]:
# Calculator de Storage

def estimate_storage(
    events_per_day: int,
    avg_event_size_bytes: int,
    retention_days: int,
    compression_ratio: float = 0.1,  # Parquet/Delta = ~10x compression
    replication_factor: int = 1,
    growth_rate_monthly: float = 0.1  # 10% growth/month
) -> dict:
    """
    Estime le storage nécessaire pour un Data Lake.
    """
    # Calcul de base
    raw_daily_gb = (events_per_day * avg_event_size_bytes) / (1024**3)
    compressed_daily_gb = raw_daily_gb * compression_ratio
    
    # Avec retention
    total_gb = compressed_daily_gb * retention_days * replication_factor
    
    # Projection 1 an avec croissance
    monthly_growth = 1 + growth_rate_monthly
    total_1year_gb = total_gb * (monthly_growth ** 12)
    
    return {
        "daily_raw_gb": round(raw_daily_gb, 2),
        "daily_compressed_gb": round(compressed_daily_gb, 2),
        "total_current_gb": round(total_gb, 2),
        "total_1year_gb": round(total_1year_gb, 2),
        "monthly_cost_s3_standard": round(total_gb * 0.023, 2),  # $0.023/GB
        "monthly_cost_1year": round(total_1year_gb * 0.023, 2),
    }

# Exemple : E-commerce
result = estimate_storage(
    events_per_day=10_000_000,      # 10M events/jour
    avg_event_size_bytes=500,        # 500 bytes/event (JSON)
    retention_days=365,              # 1 an de rétention
    compression_ratio=0.1,           # Parquet compresse ~10x
    growth_rate_monthly=0.15         # 15% croissance/mois
)

print("📊 ESTIMATION STORAGE")
print("="*50)
print(f"  Daily raw data      : {result['daily_raw_gb']} GB")
print(f"  Daily compressed    : {result['daily_compressed_gb']} GB")
print(f"  Total current       : {result['total_current_gb']} GB ({result['total_current_gb']/1024:.1f} TB)")
print(f"  Total in 1 year     : {result['total_1year_gb']} GB ({result['total_1year_gb']/1024:.1f} TB)")
print(f"\n💰 COÛT S3 (Standard)")
print(f"  Current monthly     : ${result['monthly_cost_s3_standard']}")
print(f"  In 1 year monthly   : ${result['monthly_cost_1year']}")

### 5.2 Estimer le Compute

In [ ]:
# Calculator de Compute Spark

def estimate_spark_cluster(
    data_to_process_gb: float,
    processing_time_target_hours: float,
    complexity_factor: float = 1.0,  # 1.0 = simple ETL, 2.0 = joins complexes, 3.0 = ML
    memory_per_gb_data: float = 2.0,  # 2GB RAM pour 1GB data (rule of thumb)
) -> dict:
    """
    Estime la taille d'un cluster Spark.
    """
    # Mémoire totale nécessaire
    total_memory_gb = data_to_process_gb * memory_per_gb_data * complexity_factor
    
    # Throughput cible
    throughput_gb_per_hour = data_to_process_gb / processing_time_target_hours
    
    # Sizing nodes (ex: r5.2xlarge = 64GB RAM, 8 cores)
    node_memory_gb = 64
    node_cores = 8
    num_nodes = max(1, int(total_memory_gb / (node_memory_gb * 0.75)))  # 75% usable
    
    # Coût (r5.2xlarge on-demand = ~$0.50/hour)
    hourly_cost = num_nodes * 0.50
    job_cost = hourly_cost * processing_time_target_hours
    
    return {
        "total_memory_needed_gb": round(total_memory_gb, 0),
        "throughput_gb_per_hour": round(throughput_gb_per_hour, 1),
        "recommended_nodes": num_nodes,
        "total_cores": num_nodes * node_cores,
        "total_memory_gb": num_nodes * node_memory_gb,
        "hourly_cost": round(hourly_cost, 2),
        "job_cost": round(job_cost, 2),
    }

# Exemple : Job ETL quotidien
result = estimate_spark_cluster(
    data_to_process_gb=500,           # 500 GB à traiter
    processing_time_target_hours=2,    # En 2 heures max
    complexity_factor=1.5,             # ETL avec quelques joins
)

print("🖥️ ESTIMATION CLUSTER SPARK")
print("="*50)
print(f"  Memory needed       : {result['total_memory_needed_gb']} GB")
print(f"  Throughput target   : {result['throughput_gb_per_hour']} GB/hour")
print(f"\n📊 CLUSTER SIZING (r5.2xlarge)")
print(f"  Nodes recommended   : {result['recommended_nodes']}")
print(f"  Total cores         : {result['total_cores']}")
print(f"  Total memory        : {result['total_memory_gb']} GB")
print(f"\n💰 COÛT")
print(f"  Hourly              : ${result['hourly_cost']}")
print(f"  Per job             : ${result['job_cost']}")
print(f"  Monthly (1 job/day) : ${result['job_cost'] * 30:.0f}")

### 5.3 Règles de Capacity Planning

```text
┌─────────────────────────────────────────────────────────────────────────────┐
│                    RÈGLES DE SIZING                                         │
│                                                                             │
│   STORAGE                                                                   │
│   ───────                                                                   │
│   • Prévoir 2-3x le volume actuel pour la croissance                       │
│   • Parquet/Delta = 5-10x compression vs JSON/CSV                          │
│   • Ajouter 20% pour les metadata, logs, temp files                        │
│                                                                             │
│   SPARK MEMORY                                                              │
│   ────────────                                                              │
│   • 2-4 GB RAM par 1 GB de données à traiter                               │
│   • Joins/aggregations complexes = x2 ou x3                                │
│   • Broadcast joins si une table < 10 MB                                   │
│                                                                             │
│   SPARK PARALLELISM                                                         │
│   ─────────────────                                                         │
│   • 2-4 partitions par core                                                │
│   • Partition size idéale = 128 MB - 256 MB                                │
│   • Trop de partitions = overhead scheduling                               │
│                                                                             │
│   KAFKA                                                                     │
│   ─────                                                                     │
│   • Partitions = throughput / 10 MB/s (par partition)                      │
│   • Replication factor = 3 (production)                                    │
│   • Retention = jours nécessaires pour replay                              │
│                                                                             │
│   CLICKHOUSE                                                                │
│   ──────────                                                                │
│   • 1 core = ~100-500 MB/s scan rate                                       │
│   • RAM = 10-20% du dataset fréquemment queryé                             │
└─────────────────────────────────────────────────────────────────────────────┘
```

---

## 6. Anti-Patterns à Éviter

### 6.1 Resume-Driven Development

```text
┌─────────────────────────────────────────────────────────────────────────────┐
│                    ❌ RESUME-DRIVEN DEVELOPMENT                             │
│                                                                             │
│   "On devrait utiliser Kubernetes + Kafka + Flink + Delta Lake +           │
│    GraphQL + Terraform + ArgoCD pour notre dashboard interne               │
│    qui a 10 utilisateurs."                                                 │
│                                                                             │
│   SYMPTÔMES :                                                               │
│   • Choisir une techno parce que "ça fait bien sur le CV"                  │
│   • Over-engineering pour des use cases simples                            │
│   • Stack incompréhensible par l'équipe                                    │
│   • Coûts d'infrastructure disproportionnés                                │
│                                                                             │
│   SOLUTION :                                                                │
│   • "Boring technology" d'abord                                            │
│   • Justifier CHAQUE ajout de complexité                                   │
│   • Demander : "Est-ce que PostgreSQL suffirait ?"                         │
└─────────────────────────────────────────────────────────────────────────────┘
```

### 6.2 Cargo Cult Architecture

```text
┌─────────────────────────────────────────────────────────────────────────────┐
│                    ❌ CARGO CULT ARCHITECTURE                               │
│                                                                             │
│   "Netflix utilise X, donc on devrait utiliser X."                         │
│                                                                             │
│   SYMPTÔMES :                                                               │
│   • Copier l'architecture des FAANG sans comprendre le contexte            │
│   • Ignorer les différences d'échelle (10 users vs 100M users)             │
│   • Ignorer les différences de budget et d'équipe                          │
│                                                                             │
│   RÉALITÉ :                                                                 │
│   • Netflix a des milliers d'ingénieurs                                    │
│   • Leur échelle justifie la complexité                                    │
│   • Ce qui marche pour eux ne marchera pas pour toi                        │
│                                                                             │
│   SOLUTION :                                                                │
│   • Comprendre POURQUOI ils ont fait ce choix                              │
│   • Évaluer si tes contraintes sont similaires                             │
│   • Adapter, ne pas copier aveuglément                                     │
└─────────────────────────────────────────────────────────────────────────────┘
```

### 6.3 Autres Anti-Patterns

| Anti-Pattern | Description | Solution |
|--------------|-------------|----------|
| **Premature Optimization** | Optimiser avant de mesurer | Mesurer d'abord, optimiser ensuite |
| **Golden Hammer** | "Spark pour tout !" | Choisir l'outil adapté au problème |
| **Not Invented Here** | Refuser les solutions existantes | Évaluer buy vs build objectivement |
| **Analysis Paralysis** | Trop d'analyse, pas d'action | Time-box les décisions, PoC rapides |
| **Shiny Object Syndrome** | Adopter chaque nouvelle techno | Attendre la maturité, évaluer les risques |
| **Distributed Monolith** | Microservices trop couplés | Si tout doit être déployé ensemble = monolith |

---

## 7. Case Studies

### 7.1 Startup vs Enterprise

```text
┌─────────────────────────────────────────────────────────────────────────────┐
│                    STARTUP (< 50 personnes, < $1M ARR)                      │
│                                                                             │
│   CONTRAINTES :                          ARCHITECTURE RECOMMANDÉE :         │
│   • Budget limité                        • Managed services (BigQuery,      │
│   • Équipe petite (1-3 DE)                 Snowflake, Fivetran)             │
│   • Time-to-market critique              • Pas de Kubernetes                │
│   • Besoins qui changent vite            • PostgreSQL > Data Lake           │
│                                          • dbt Cloud > dbt Core             │
│   RÈGLE : Minimiser l'ops,               • Éviter le self-hosted            │
│           maximiser la vélocité                                             │
└─────────────────────────────────────────────────────────────────────────────┘

┌─────────────────────────────────────────────────────────────────────────────┐
│                    SCALE-UP (50-500 personnes, $10M+ ARR)                   │
│                                                                             │
│   CONTRAINTES :                          ARCHITECTURE RECOMMANDÉE :         │
│   • Volume croissant rapidement          • Lakehouse (Delta/Iceberg)        │
│   • Équipe data 5-20 personnes           • Spark + Kubernetes               │
│   • Besoin de gouvernance                • Airflow managed ou K8s           │
│   • Multi-équipes consomment data        • Data Catalog (DataHub)           │
│                                          • Data Contracts                   │
│   RÈGLE : Investir dans la plateforme,                                      │
│           mais rester pragmatique                                           │
└─────────────────────────────────────────────────────────────────────────────┘

┌─────────────────────────────────────────────────────────────────────────────┐
│                    ENTERPRISE (500+ personnes)                              │
│                                                                             │
│   CONTRAINTES :                          ARCHITECTURE RECOMMANDÉE :         │
│   • Legacy systems à intégrer            • Data Mesh                        │
│   • Compliance stricte (GDPR, SOX)       • Platform team dédié              │
│   • Équipes data 50+ personnes           • Self-serve analytics             │
│   • Multi-cloud possible                 • Strong governance                │
│   • Budgets importants                   • Lineage & cataloging             │
│                                          • Hybrid cloud                     │
│   RÈGLE : Gouvernance + autonomie                                           │
│           des domaines                                                      │
└─────────────────────────────────────────────────────────────────────────────┘
```

### 7.2 Case Study : E-Commerce

```text
┌─────────────────────────────────────────────────────────────────────────────┐
│                    CASE STUDY : E-COMMERCE PLATFORM                         │
│                                                                             │
│   CONTEXTE :                                                                │
│   • 1M commandes/jour                                                       │
│   • 50M products                                                            │
│   • Besoin : Real-time inventory, recommendations, fraud detection         │
│                                                                             │
│   ARCHITECTURE :                                                            │
│                                                                             │
│   ┌─────────────┐     ┌─────────┐     ┌─────────────┐                      │
│   │   Sources   │────▶│  Kafka  │────▶│   Flink     │──┬──▶ Redis (reco)  │
│   │ (orders,    │     │         │     │ (streaming) │  │                   │
│   │  clicks,    │     └────┬────┘     └─────────────┘  └──▶ Alerts (fraud)│
│   │  inventory) │          │                                               │
│   └─────────────┘          │          ┌─────────────┐     ┌─────────────┐ │
│                            └─────────▶│   Spark     │────▶│ Delta Lake  │ │
│                                       │   (batch)   │     │ (Lakehouse) │ │
│                                       └─────────────┘     └──────┬──────┘ │
│                                                                  │        │
│                                                           ┌──────▼──────┐ │
│                                                           │  ClickHouse │ │
│                                                           │ (dashboards)│ │
│                                                           └─────────────┘ │
│                                                                            │
│   DÉCISIONS CLÉS :                                                         │
│   • Kafka : source of truth pour events                                   │
│   • Flink : real-time pour fraud (< 1s latency)                           │
│   • Spark : batch pour ML training, heavy transformations                 │
│   • ClickHouse : OLAP pour dashboards (vs query Delta direct)             │
└─────────────────────────────────────────────────────────────────────────────┘
```

### 7.3 Case Study : FinTech

```text
┌─────────────────────────────────────────────────────────────────────────────┐
│                    CASE STUDY : FINTECH (Trading Platform)                  │
│                                                                             │
│   CONTEXTE :                                                                │
│   • Compliance stricte (audit, GDPR)                                        │
│   • Low latency critique (< 100ms)                                          │
│   • Data retention 7 ans                                                    │
│   • Exactitude 100% requise                                                 │
│                                                                             │
│   DÉCISIONS SPÉCIFIQUES :                                                   │
│                                                                             │
│   1. IMMUTABILITÉ                                                           │
│      • Append-only tables (jamais update/delete)                           │
│      • Event sourcing pattern                                               │
│      • Full audit trail                                                     │
│                                                                             │
│   2. EXACTITUDE                                                             │
│      • Decimal types (jamais float pour l'argent !)                        │
│      • Idempotency sur tous les pipelines                                  │
│      • Reconciliation batch quotidienne                                     │
│                                                                             │
│   3. RETENTION                                                              │
│      • Hot storage : 30 jours (SSD)                                        │
│      • Warm storage : 1 an (HDD)                                           │
│      • Cold storage : 7 ans (S3 Glacier)                                   │
│      • Lifecycle policies automatisées                                      │
│                                                                             │
│   4. COMPLIANCE                                                             │
│      • Data Catalog obligatoire                                            │
│      • PII tagging automatique                                              │
│      • Access logs + alerts                                                 │
└─────────────────────────────────────────────────────────────────────────────┘
```

---

## 8. Checklist Architecture Review

Utilise cette checklist pour valider une architecture avant de l'implémenter :

In [ ]:
# Checklist Architecture Review

checklist = """
# ═══════════════════════════════════════════════════════════════
# ARCHITECTURE REVIEW CHECKLIST
# ═══════════════════════════════════════════════════════════════

## 1. PROBLEM DEFINITION
- [ ] Le problème business est clairement défini
- [ ] Les requirements sont documentés
- [ ] Les success criteria sont mesurables
- [ ] Les contraintes sont identifiées (budget, temps, équipe)

## 2. SOLUTION FIT
- [ ] La solution résout le problème (pas plus, pas moins)
- [ ] Les alternatives ont été considérées
- [ ] Une solution plus simple a été évaluée
- [ ] L'ADR est rédigé

## 3. SCALABILITÉ
- [ ] Volume actuel supporté
- [ ] Croissance 1 an anticipée
- [ ] Croissance 3 ans anticipée
- [ ] Bottlenecks identifiés et plan de mitigation

## 4. RELIABILITY
- [ ] SLA défini et réaliste
- [ ] Single points of failure identifiés
- [ ] Disaster recovery plan
- [ ] Monitoring et alerting prévu

## 5. OPÉRABILITÉ
- [ ] L'équipe a les compétences nécessaires
- [ ] Documentation prévue
- [ ] Runbooks pour incidents
- [ ] On-call rotation définie

## 6. COÛTS
- [ ] Coûts estimés (storage, compute, licenses)
- [ ] Budget approuvé
- [ ] Optimisations identifiées (spot instances, reserved, etc.)
- [ ] Alertes coûts configurées

## 7. SÉCURITÉ
- [ ] Données sensibles identifiées
- [ ] Encryption at rest et in transit
- [ ] Access control (RBAC)
- [ ] Audit logging

## 8. ÉVOLUTIVITÉ
- [ ] Architecture modulaire
- [ ] Pas de vendor lock-in critique
- [ ] Migration possible si besoin
- [ ] Backward compatibility considérée

## VALIDATION FINALE
- [ ] Review par un pair
- [ ] Stakeholders informés
- [ ] Go/No-Go decision
"""

print(checklist)

---

## 9. Exercices Pratiques

### Exercice 1 : Analyse de Requirements

Une startup de livraison de repas te demande de concevoir leur data platform :
- 50K commandes/jour, croissance 20%/mois
- Besoin de dashboards temps réel pour les opérations
- Équipe : 2 data engineers (expérience Python/SQL)
- Budget : $5K/mois max
- Timeline : 2 mois pour le MVP

**Questions :**
1. Quels sont les requirements fonctionnels et non-fonctionnels ?
2. Quelle architecture recommandes-tu ? Pourquoi ?
3. Quels outils spécifiques ?
4. Quels sont les risques et mitigations ?

---

### Exercice 2 : ADR Writing

Rédige un ADR complet pour le choix suivant :
> "Utiliser Airflow plutôt que Prefect pour l'orchestration"

Inclus : context, options considérées, décision, conséquences.

---

### Exercice 3 : Capacity Planning

Une plateforme IoT génère :
- 10,000 devices
- Chaque device envoie 1 event/seconde
- Chaque event = 200 bytes
- Retention : 90 jours

Calcule :
1. Volume quotidien (raw et compressé)
2. Volume total avec retention
3. Coût S3 mensuel estimé
4. Sizing Kafka (partitions recommandées)

---

### Exercice 4 : Anti-Pattern Detection

Analyse cette proposition d'architecture et identifie les anti-patterns :

> "Pour notre app interne avec 50 utilisateurs, je propose :
> - Kubernetes cluster avec 10 nodes
> - Kafka avec 100 partitions
> - Flink pour le streaming
> - Cassandra pour le stockage
> - Elasticsearch pour le search
> - Custom ML pipeline from scratch"

---

### Exercice 5 : Case Study Analysis

Compare les architectures data de :
1. Uber (https://eng.uber.com/)
2. Airbnb (https://medium.com/airbnb-engineering)
3. Spotify (https://engineering.atspotify.com/)

Pour chaque :
- Quels sont leurs principaux défis data ?
- Quelles solutions ont-ils adoptées ?
- Qu'est-ce qui est applicable à une entreprise plus petite ?

---

## 📚 Ressources

### Articles & Blogs
- [Designing Data-Intensive Applications](https://dataintensive.net/) — Martin Kleppmann (THE book)
- [Architecture of Open Source Applications](https://aosabook.org/)
- [High Scalability Blog](http://highscalability.com/)
- [Netflix Tech Blog](https://netflixtechblog.com/)

### ADR Resources
- [ADR GitHub](https://adr.github.io/)
- [Documenting Architecture Decisions](https://cognitect.com/blog/2011/11/15/documenting-architecture-decisions)

### Architecture Patterns
- [Lambda Architecture](http://lambda-architecture.net/)
- [Kappa Architecture](https://www.oreilly.com/radar/questioning-the-lambda-architecture/)
- [Data Mesh Principles](https://martinfowler.com/articles/data-mesh-principles.html)

---

## ➡️ Prochaine étape

👉 **Module suivant : `35_leadership_tradeoffs`** — Leadership & Trade-offs in Data Engineering

---

🎉 **Félicitations !** Tu maîtrises maintenant les patterns d'architecture et la prise de décision.